In [1]:
import sys
sys.path.insert(0, r'C:\Users\Public\WPy64-39100\notebooks\Anjana Shaji')
import pandas as pd
import pensionpro_api as pp
import json
import datetime
import os
import time
import openpyxl
start = datetime.datetime.now()
print(start)

2024-07-19 09:53:48.030885


In [2]:
d12_df = pd.read_table(r'F:\ASC\USER\All ASC Cases - D12.TXT',index_col=None).astype(object)
d18_df = pd.read_table(r'F:\ASC\USER\All ASC Cases - D18.TXT',index_col=None).astype(object)

d12_df.rename(columns={'S:CLIENTNO': 'tpa_plan_id'}, inplace=True)
d18_df.rename(columns={'S:CLIENTNO': 'tpa_plan_id'}, inplace=True)
output_folder = r'Y:\ASC\Exported Reports\SSAPlanList.txt'
asc_file_folder = r'Y:\ASC\Exported Reports\SSAs'
d18_df

,$LIBKEY,$LIBID,$LIBDESC,$PLANKEY,tpa_plan_id
0,8,D18,Active Plans II,1,2738
1,8,D18,Active Plans II,2,5699
2,8,D18,Active Plans II,3,3858
3,8,D18,Active Plans II,4,5663
4,8,D18,Active Plans II,10,9308
...,...,...,...,...,...
5253,8,D18,Active Plans II,6160,6172-22
5254,8,D18,Active Plans II,6161,6268
5255,8,D18,Active Plans II,6162,20570
5256,8,D18,Active Plans II,6163,20570


In [3]:
# Fired - Pending Final Services - 12879, Pending Termination - 64622, Terminated - 12689
plan_startus_id_list = [12879, 64622, 12689]
error_path = r'Y:\Automation\Team Scripts\Anjana Shaji\8955 SSA DF\error_report.xlsx'
cols = ['TPA Plan ID', 'ASC Client number Missing', 'ASC Text File Missing', 'Project Closed Out', 'DGEM Upload Error', 'DGEM Data Missing', 'Validation Error', 'Form Downloaded', 'Delivered to Client']
df_error = pd.DataFrame(columns=cols)
df_error.to_excel(error_path, index=False)

In [4]:
def launch_project(tpa_id, period_end_date, period_start_date):
    project_name = "Form 8955-SSA (Automated)"
    data_json = pp.get_project_template_by_name(project_name)
    project_template_id = data_json[0]["Id"]
    show_on_PSL = True
    response_data = pp.add_project(planid = tpa_id, ProjectTemplateId = project_template_id, PeriodStart = period_start_date, PeriodEnd = period_end_date, EnableToShowOnPSLProjectsTab = show_on_PSL)
    print('Project launched successfully for plan: ', tpa_id)

In [5]:
def pad_with_zeros(string):
    string = str(string)
    if len(string) < 4:
        zeros_to_add = 4 - len(string)
        padded_string = '0' * zeros_to_add + string
        return padded_string
    else:
        return string

In [6]:
def check_if_asc_file_exists(plan_id, period_end_date, asc_file_list):
    found_flag = False
    file_name = None
    for file in asc_file_list:
        file_plan_id = file.split("_")[0]
        file_period_end = file.split("_")[1]
        file_period_end_dt = datetime.datetime.strptime(file_period_end,'%m%d%Y')
        file_period_end_dt = file_period_end_dt + datetime.timedelta(hours=12)
        if file_plan_id == plan_id and file_period_end_dt == period_end_date:
            found_flag = True
            file_name = file
    return found_flag, file_name 

In [7]:
def create_file_for_asc_wiz(asc_target):
    # Find ASC client id using TPA Plan ID.
    asc_client_id_list = []
    df_error_log = pd.read_excel(error_path)
    for plan_id in asc_target:
        try:
            plan_id = str(plan_id)
            if plan_id in d12_df['tpa_plan_id'].to_list():
                print("D12", plan_id)
                asc_client_id = d12_df.loc[d12_df['tpa_plan_id'] == plan_id, '$PLANKEY'].values[0]
                asc_client_id = pad_with_zeros(asc_client_id)
                print('asc_client_id: ',asc_client_id)
                text = f'D12:{asc_client_id}'
                asc_client_id_list.append(text)
            else:
                print("D18", plan_id)
                asc_client_id = d18_df.loc[d18_df['tpa_plan_id'] == plan_id, '$PLANKEY'].values[0]
                asc_client_id = pad_with_zeros(asc_client_id)
                print('asc_client_id: ',asc_client_id)
                text = f'D18:{asc_client_id}'
                asc_client_id_list.append(text)
        except Exception as e:
            print(e)
            projects = pp.get_projects_by_planid(plan_id,filters=f"Name eq 'Form 8955-SSA (Automated)'")
            project = [project for project in projects if project['CompletedOn'] is None]
            if len(project) > 0:
                project = project[0]
            project_id = project['Id']
            
            task_group = pp.get_task_groups_by_projectid(project_id, expand = 'Tasks.Taskitems')[0]
            for task in task_group['Tasks']:
                if task['TaskName'] == 'Completion - Form 8955-SSA':
                    task_id = task['Id']
                    for task_item in task['TaskItems']:
                        if task_item['ShortName'] == '8955 Validation Error' and task_item['Value'] == None:
                            task_item['Value'] = 'Complete'
                            pp.update_taskitem(task_item)
                            note_text = 'ASC equivalent client number missing in D12 and D18'
                            payload = {
                                "ProjectID": project_id, 
                                "NoteText": f"{note_text}",
                                "NoteCategoryId": 3514,
                                "ShowOnPSL": False
                                    }
                
                            x = pp.add_note(payload)
                        elif task_item['ShortName'] == '8955 Validation Error' and task_item['Value'] == 'Complete':
                            pp.override_task(task_id)
            df_error_log.loc[df_error_log['TPA Plan ID'].astype(str).str.contains(plan_id), 'ASC Client number Missing'] = 'Yes'
            continue
    print(df_error_log)
    df_error_log.to_excel(error_path, index = False)        
    with open(output_folder, 'w') as f:
        for client_id in asc_client_id_list:
            f.write(f"{client_id}\n")    

In [8]:
def get_asc_files():
    asc_files = os.listdir(asc_file_folder)
    asc_file_list = [file for file in asc_files if 'FormSSA' in file and file.endswith('.txt') and not file.startswith('_')]
    return asc_file_list

In [9]:
def get_ssa_and_asc_df(plan_dict):
    file_found_index = []
    file_not_found_index = []
    ssa_target = []
    asc_target = []
    plan_id_dict_asc = {}
    for plan_id in plan_dict:
        period_end_date = plan_dict[plan_id]
        period_end_date = datetime.datetime.strptime(period_end_date,'%m/%d/%Y %H:%M:%S %p')
        asc_file_list = get_asc_files()
        file_exists_flag, file_name = check_if_asc_file_exists(plan_id, period_end_date, asc_file_list)
        if file_exists_flag:
            print('ASC file found for plan: ', plan_id)
            plan_id_dict_asc[plan_id] = file_name
            ssa_target.append(plan_id)
        else:
            print('ASC file not found for plan: ', plan_id)
            asc_target.append(plan_id)
    return ssa_target, asc_target, plan_id_dict_asc

In [10]:
#do not run if project is launched by Jason
today = datetime.date.today()
two_days_ago = today - datetime.timedelta(days = 2)
two_days_ago = datetime.datetime.strftime(two_days_ago, "%Y-%m-%d")
print(two_days_ago)
expand = 'TaskGroup.Project'
filters = f"(TaskName eq 'Filing Invitation' and (TaskGroup.Project.Name eq 'DC Annual Governmental Forms - Schedule I' or TaskGroup.Project.Name eq 'DC Annual Governmental Forms - Small Filer (Automated)')  and DateCompleted ge '{two_days_ago}') or (TaskName eq 'Client Communications' and TaskGroup.Project.Name eq 'DC Annual Governmental Forms - Audit' and DateCompleted ge '{two_days_ago}')"
tasks = pp.get_tasks(filters=filters, expand=expand, get_all=True)
len(tasks)

2024-07-17


204

In [11]:
#do not run if project is launched by Jason
today = datetime.date.today()
yesterday = today - datetime.timedelta(days = 1)
yesterday = datetime.datetime.strftime(yesterday, "%Y-%m-%d")
print(yesterday)
i = 0
for task in tasks:
    date_completed = task['DateCompleted']
    date_completed = date_completed.rsplit('T')[0]
    # print(date_completed)
    if date_completed == yesterday:
        i = i+1
print(i)

2024-07-18
81


In [12]:
#do not run if project is launched by Jason
plan_id_dict = {}
today = datetime.date.today()
yesterday = today - datetime.timedelta(days = 1)
yesterday = datetime.datetime.strftime(yesterday, "%Y-%m-%d")
for task in tasks:
    date_completed = task['DateCompleted']
    date_completed = date_completed.rsplit('T')[0]
    if date_completed == yesterday:
        project = task['TaskGroup']['Project']
        internal_plan_id = project['PlanId']
        period_end_date = project['PeriodEnd']
        period_start_date = project['PeriodStart']
        plan = pp.get_plan_by_planid(internal_plan_id)
        tpa_id = plan['InternalPlanId']
        plan_status_id = plan['PlanStatusId']
        try:
            if plan_status_id in plan_startus_id_list:
                print('Yes')
                terminated_on_date = plan['TerminatedOn']
                print(terminated_on_date)
                if terminated_on_date < period_end_date:
                    print('Plan is exempted from launching project form 8955')
                    continue
                else:
                    project_data = pp.get_projects_by_planid(tpa_id, filters= f"Name eq 'Form 8955-SSA (Automated)' and PeriodEnd eq '{period_end_date}'")
                    if not project_data:
                        plan_id_dict[tpa_id] = period_end_date
                        launch_project(tpa_id, period_end_date, period_start_date) 
                    else:
                        print(f'Project form 8955 already found for plan: {tpa_id}')
            else:
                print('in else')
                project_data = pp.get_projects_by_planid(tpa_id, filters= f"Name eq 'Form 8955-SSA (Automated)' and PeriodEnd eq '{period_end_date}'")
                if not project_data:
                    plan_id_dict[tpa_id] = period_end_date
                    launch_project(tpa_id, period_end_date, period_start_date) 
                else:
                    print(f'Project form 8955 already found for plan: {tpa_id}')
        except Exception as e:
            print(e)
            project_data = pp.get_projects_by_planid(tpa_id, filters= f"Name eq 'Form 8955-SSA (Automated)' and PeriodEnd eq '{period_end_date}'")
            if not project_data:
                plan_id_dict[tpa_id] = period_end_date
                try:
                    launch_project(tpa_id, period_end_date, period_start_date) 
                except Exception as e:
                    print(e)
                    continue
            else:
                print(f'Project form 8955 already found for plan: {tpa_id}')
            continue
            
print('Done!')

in else
Project launched successfully for plan:  9766
Yes
6/30/2023 12:00:00 AM
Project launched successfully for plan:  19950
Yes
3/31/2024 12:00:00 AM
Project launched successfully for plan:  12441
in else
Project form 8955 already found for plan: 17059
in else
Project form 8955 already found for plan: 12670
in else
Project launched successfully for plan:  16177
in else
Project form 8955 already found for plan: 18844
in else
Project form 8955 already found for plan: 11303
in else
Project launched successfully for plan:  11432
in else
Project launched successfully for plan:  10681
in else
Project launched successfully for plan:  11296
in else
Project launched successfully for plan:  11575
in else
Project form 8955 already found for plan: 90393
in else
Project launched successfully for plan:  11078
in else
Project form 8955 already found for plan: 5619
in else
Project launched successfully for plan:  11996
in else
Project launched successfully for plan:  4143
in else
Project launched s

In [13]:
#needs to be removed once the bulk launch is completed by Jason 
df = pp.get_worktray('Automation', get_all=True)
filt1 = df['task_name'] == 'Completion - Form 8955-SSA'
filt2 = df['proj_name'] == 'Form 8955-SSA (Automated)'
df = df[filt1 & filt2]
print(len(df))

744


In [14]:
df_error.dtypes

TPA Plan ID                  object
ASC Client number Missing    object
ASC Text File Missing        object
Project Closed Out           object
DGEM Upload Error            object
DGEM Data Missing            object
Validation Error             object
Form Downloaded              object
Delivered to Client          object
Filed                        object
dtype: object

In [14]:
plan_id_dict = {}
plan_id_list = []
for index, row in df.iterrows():
        plan_id =  row['planid']
        period_end_date = row['per_end']
        plan_id_dict[plan_id] = period_end_date
        plan_id_list.append(plan_id)
df_error_log = pd.read_excel(error_path)
df_error_log['TPA Plan ID'] = plan_id_list
df_error_log.to_excel(error_path, index = False)
plan_id_dict

{'3485': '12/31/2023 12:00:00 AM',
 '10457': '1/31/2024 12:00:00 AM',
 '16202': '12/31/2023 12:00:00 AM',
 '18265': '12/31/2023 12:00:00 AM',
 '4658': '12/31/2023 12:00:00 AM',
 '18355': '12/31/2023 12:00:00 AM',
 '3517': '3/31/2024 12:00:00 AM',
 '17720': '12/31/2023 12:00:00 AM',
 '4537': '12/31/2023 12:00:00 AM',
 '19426': '12/31/2023 12:00:00 AM',
 '90697': '3/31/2024 12:00:00 AM',
 '17451': '12/31/2023 12:00:00 AM',
 '18666': '2/29/2024 12:00:00 AM',
 '5908': '12/31/2023 12:00:00 AM',
 '90448': '12/31/2023 12:00:00 AM',
 '16368': '1/31/2024 12:00:00 AM',
 '12161': '4/30/2024 12:00:00 AM',
 '17690': '3/31/2024 12:00:00 AM',
 '16559': '4/30/2024 12:00:00 AM',
 '15394': '4/30/2024 12:00:00 AM',
 '16801': '12/31/2023 12:00:00 AM',
 '19637': '12/31/2023 12:00:00 AM',
 '20102': '12/31/2023 12:00:00 AM',
 '12263': '12/31/2023 12:00:00 AM',
 '9880': '3/31/2024 12:00:00 AM',
 '90567': '3/31/2024 12:00:00 AM',
 '10292': '3/31/2024 12:00:00 AM',
 '16462': '12/31/2023 12:00:00 AM',
 '10229': 

In [15]:
len(plan_id_dict)

741

In [16]:
create_file_for_asc_wiz(plan_id_dict)

D18 3485
index 0 is out of bounds for axis 0 with size 0


C:\Users\ashaji\AppData\Local\Temp\ipykernel_9792\3389629740.py:49: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Yes' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_error_log.loc[df_error_log['TPA Plan ID'].astype(str).str.contains(plan_id), 'ASC Client number Missing'] = 'Yes'


D12 10457
asc_client_id:  8347
D18 16202
index 0 is out of bounds for axis 0 with size 0
D18 18265
index 0 is out of bounds for axis 0 with size 0
D18 4658
index 0 is out of bounds for axis 0 with size 0
D18 18355
asc_client_id:  5302
D12 3517
asc_client_id:  4067
D18 17720
asc_client_id:  2413
D18 4537
index 0 is out of bounds for axis 0 with size 0
D18 19426
asc_client_id:  4268
D12 90697
asc_client_id:  8073
D18 17451
asc_client_id:  2558
D18 18666
asc_client_id:  3663
D18 5908
index 0 is out of bounds for axis 0 with size 0
D12 90448
asc_client_id:  8020
D18 16368
asc_client_id:  1584
D18 12161
asc_client_id:  1965
D18 17690
asc_client_id:  2009
D18 16559
asc_client_id:  2301
D18 15394
asc_client_id:  0345
D18 16801
asc_client_id:  1829
D18 19637
index 0 is out of bounds for axis 0 with size 0
D18 20102
asc_client_id:  4903
D18 12263
asc_client_id:  0272
D12 9880
asc_client_id:  2112
D12 90567
asc_client_id:  8041
D12 10292
asc_client_id:  7814
D18 16462
asc_client_id:  2395
D12 10

In [33]:
df_error_log

,Unnamed: 0,TPA Plan ID,ASC Client number Missing,ASC Text File Missing,Project Closed Out,DGEM Upload Error,DGEM Data Missing,Validation Error,Form Downloaded,Delivered to Client,Filed
0,NaN,3485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,10457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,16202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,11978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,5363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,5556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,5773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,4831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
670,NaN,9503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
done_file_flag = False
while not done_file_flag:
    asc_files = os.listdir(asc_file_folder)
    asc_file_list = [file for file in asc_files if 'Done_' in file and file.endswith('.txt')]
    if asc_file_list:
        print('Done file found!!!')
        asc_file = asc_file_list[0]
        done_file_flag = True
        path = f'{asc_file_folder}\{asc_file}'
        os.remove(path)
        print('Removed done file')
    else:
        print('Done file not found yet, waiting!!!')
        time.sleep(600)


Done file not found yet, waiting!!!


In [21]:
end = datetime.datetime.now()
diff = end - start 
minutes = diff.total_seconds() / 60
print('minutes: ',minutes)

minutes:  111.10953401666667
